In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

from custom_models import get_cell_based_tiny_net

# NB201
from nas_201_api import NASBench201API as API

2022-10-20 06:01:20.422830: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
parser = argparse.ArgumentParser("Random search for NAS.")
parser.add_argument("--data_path", type=str, default='../cifar.python', help="The path to dataset")
parser.add_argument("--dataset", type=str, default='cifar10',choices=["cifar10", "cifar100", "ImageNet16-120"], help="Choose between Cifar10/100 and ImageNet-16.")

# channels and number-of-cells
parser.add_argument("--search_space_name", type=str, default='nas-bench-201', help="The search space name.")
parser.add_argument("--config_path", type=str, default='./MY.config', help="The path to the configuration.")
parser.add_argument("--max_nodes", type=int, default=4, help="The maximum number of nodes.")
parser.add_argument("--channel", type=int, default=16, help="The number of channels.")
parser.add_argument("--num_cells", type=int, default=5, help="The number of cells in one stage.")
parser.add_argument("--select_num", type=int, default=100, help="The number of selected architectures to evaluate.")
parser.add_argument("--track_running_stats", type=int, default=0, choices=[0, 1], help="Whether use track_running_stats or not in the BN layer.")
# log
parser.add_argument("--workers", type=int, default=4, help="number of data loading workers")
parser.add_argument("--save_dir", type=str, default='./tmp', help="Folder to save checkpoints and log.")
# parser.add_argument("--arch_nas_dataset", type=str, default='../NAS-Bench-201-v1_1-096897.pth', help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--arch_nas_dataset", type=str, default=None, help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--print_freq", type=int, default=100, help="print frequency (default: 200)")
parser.add_argument("--rand_seed", type=int, default=None, help="manual seed")
args = parser.parse_args(args=[])
if args.rand_seed is None or args.rand_seed < 0:
    args.rand_seed = random.randint(1, 100000)

    
print(args.rand_seed)
print(args)
xargs=args

27924
Namespace(arch_nas_dataset=None, channel=16, config_path='./MY.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=100, rand_seed=27924, save_dir='./tmp', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [3]:
assert torch.cuda.is_available(), "CUDA is not available."
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads(xargs.workers)
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

Main Function with logger : Logger(dir=tmp, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ./MY.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 100
rand_seed        : 27924
save_dir         : ./tmp
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None


In [4]:
def search_func(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the weights
        ### sample a cell-wise randomized network
        network.module.random_genotype_per_cell(True)
        ###
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg


def valid_func(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

            network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg


def search_find_best(xloader, network, n_samples):
    with torch.no_grad():
        network.eval()
        archs, valid_accs = [], []
        # print ('obtain the top-{:} architectures'.format(n_samples))
        loader_iter = iter(xloader)
        for i in range(n_samples):
            arch = network.module.random_genotype_per_cell(True)
            try:
                inputs, targets = next(loader_iter)
            except:
                loader_iter = iter(xloader)
                inputs, targets = next(loader_iter)

            _, logits = network(inputs)
            val_top1, val_top5 = obtain_accuracy(
                logits.cpu().data, targets.data, topk=(1, 5)
            )

            archs.append(arch)
            valid_accs.append(val_top1.item())

        best_idx = np.argmax(valid_accs)
        best_arch, best_valid_acc = archs[best_idx], valid_accs[best_idx]
        return best_arch, best_valid_acc

In [5]:
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(xargs.config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.test_batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": bool(xargs.track_running_stats),
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log("{:} create API = {:} done".format(time_string(), api))

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

if last_info.exists():  # automatically resume from previous checkpoint
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start".format(last_info)
    )
    last_info = torch.load(last_info)
    start_epoch = last_info["epoch"]
    checkpoint = torch.load(last_info["last_checkpoint"])
    genotypes = checkpoint["genotypes"]
    valid_accuracies = checkpoint["valid_accuracies"]
    search_model.load_state_dict(checkpoint["search_model"])
    w_scheduler.load_state_dict(checkpoint["w_scheduler"])
    w_optimizer.load_state_dict(checkpoint["w_optimizer"])
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(
            last_info, start_epoch
        )
    )
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Files already downloaded and verified
Files already downloaded and verified
./MY.config
Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=250, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
||||||| cifar10    ||||||| Search-Loader-Num=391, Valid-Loader-Num=49, batch size=64
||||||| cifar10    ||||||| Config=Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=250, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
w-optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.025
    lr: 0.025
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)
w-scheduler : CosineAnnealingLR(warmup=0, max-epoch=250, current::epoch=0, iter=0.00, type=cosine, T-max=250, eta-min=0.001)
criterion   : CrossEntropyL

In [ ]:
# start training
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func(
        search_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func(
        valid_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    cur_arch, cur_valid_acc = search_find_best(
        valid_loader, network, xargs.select_num
    )
    logger.log(
        "[{:}] find-the-best : {:}, accuracy@1={:.2f}%".format(
            epoch_str, cur_arch, cur_valid_acc
        )
    )
    genotypes[epoch] = cur_arch
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log("\n" + "-" * 200)
logger.log("Pre-searching costs {:.1f} s".format(search_time.sum))
start_time = time.time()
best_arch, best_acc = search_find_best(valid_loader, network, xargs.select_num)
search_time.update(time.time() - start_time)
logger.log(
    "RANDOM-NAS finds the best one : {:} with accuracy={:.2f}%, with {:.1f} s.".format(
        best_arch, best_acc, search_time.sum
    )
)
if api is not None:
    logger.log("{:}".format(api.query_by_arch(best_arch, "200")))
logger.close()


[Search the 000-250-th epoch] Time Left: [00:00:00], LR=0.025
*SEARCH* [2022-10-20 06:01:43] [000-250][000/391] Time 2.90 (2.90) Data 0.17 (0.17) Base [Loss 2.344 (2.344)  Prec@1 7.81 (7.81) Prec@5 46.88 (46.88)]
*SEARCH* [2022-10-20 06:01:54] [000-250][100/391] Time 0.08 (0.14) Data 0.00 (0.00) Base [Loss 2.361 (2.318)  Prec@1 14.06 (10.04) Prec@5 46.88 (50.56)]
*SEARCH* [2022-10-20 06:02:04] [000-250][200/391] Time 0.19 (0.12) Data 0.00 (0.00) Base [Loss 2.330 (2.313)  Prec@1 10.94 (10.41) Prec@5 37.50 (51.66)]
*SEARCH* [2022-10-20 06:02:17] [000-250][300/391] Time 0.19 (0.12) Data 0.00 (0.00) Base [Loss 2.257 (2.309)  Prec@1 14.06 (10.76) Prec@5 57.81 (52.19)]
*SEARCH* [2022-10-20 06:02:29] [000-250][390/391] Time 0.12 (0.12) Data 0.00 (0.00) Base [Loss 2.330 (2.304)  Prec@1 10.00 (10.98) Prec@5 47.50 (52.71)]
[000-250] searching : loss=2.30, accuracy@1=10.98%, accuracy@5=52.71%, time-cost=48.7 s
[000-250] evaluate  : loss=2.30, accuracy@1=11.09%, accuracy@5=53.30%
[000-250] find-t

*SEARCH* [2022-10-20 06:04:19] [003-250][000/391] Time 0.49 (0.49) Data 0.35 (0.35) Base [Loss 2.168 (2.168)  Prec@1 15.62 (15.62) Prec@5 65.62 (65.62)]
*SEARCH* [2022-10-20 06:04:32] [003-250][100/391] Time 0.14 (0.13) Data 0.00 (0.00) Base [Loss 2.318 (2.267)  Prec@1 4.69 (12.55) Prec@5 48.44 (56.82)]
*SEARCH* [2022-10-20 06:04:47] [003-250][200/391] Time 0.14 (0.14) Data 0.00 (0.00) Base [Loss 2.203 (2.262)  Prec@1 21.88 (12.92) Prec@5 67.19 (57.48)]
*SEARCH* [2022-10-20 06:05:02] [003-250][300/391] Time 0.21 (0.14) Data 0.00 (0.00) Base [Loss 2.305 (2.251)  Prec@1 6.25 (13.56) Prec@5 54.69 (58.88)]
*SEARCH* [2022-10-20 06:05:15] [003-250][390/391] Time 0.14 (0.14) Data 0.00 (0.00) Base [Loss 2.304 (2.249)  Prec@1 10.00 (13.65) Prec@5 47.50 (59.11)]
[003-250] searching : loss=2.25, accuracy@1=13.65%, accuracy@5=59.11%, time-cost=171.1 s
[003-250] evaluate  : loss=2.24, accuracy@1=13.76%, accuracy@5=60.13%
[003-250] find-the-best : [Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_1

*SEARCH* [2022-10-20 06:07:26] [006-250][000/391] Time 0.40 (0.40) Data 0.30 (0.30) Base [Loss 2.278 (2.278)  Prec@1 17.19 (17.19) Prec@5 60.94 (60.94)]
*SEARCH* [2022-10-20 06:07:35] [006-250][100/391] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 2.157 (2.219)  Prec@1 14.06 (15.10) Prec@5 68.75 (61.71)]
*SEARCH* [2022-10-20 06:07:45] [006-250][200/391] Time 0.09 (0.10) Data 0.00 (0.00) Base [Loss 2.094 (2.215)  Prec@1 9.38 (14.75) Prec@5 73.44 (61.82)]
*SEARCH* [2022-10-20 06:07:54] [006-250][300/391] Time 0.09 (0.10) Data 0.00 (0.00) Base [Loss 2.097 (2.208)  Prec@1 17.19 (15.17) Prec@5 76.56 (62.56)]
*SEARCH* [2022-10-20 06:08:03] [006-250][390/391] Time 0.08 (0.09) Data 0.00 (0.00) Base [Loss 2.084 (2.211)  Prec@1 20.00 (15.21) Prec@5 70.00 (62.60)]
[006-250] searching : loss=2.21, accuracy@1=15.21%, accuracy@5=62.60%, time-cost=294.9 s
[006-250] evaluate  : loss=2.21, accuracy@1=14.80%, accuracy@5=63.13%
[006-250] find-the-best : [Structure(4 nodes with |none~0|+|skip_connect~0|no

*SEARCH* [2022-10-20 06:10:08] [009-250][000/391] Time 0.37 (0.37) Data 0.28 (0.28) Base [Loss 2.261 (2.261)  Prec@1 15.62 (15.62) Prec@5 62.50 (62.50)]
*SEARCH* [2022-10-20 06:10:16] [009-250][100/391] Time 0.13 (0.09) Data 0.00 (0.00) Base [Loss 2.211 (2.195)  Prec@1 12.50 (14.85) Prec@5 71.88 (65.83)]
*SEARCH* [2022-10-20 06:10:25] [009-250][200/391] Time 0.09 (0.09) Data 0.00 (0.00) Base [Loss 2.310 (2.191)  Prec@1 9.38 (15.30) Prec@5 46.88 (65.28)]
*SEARCH* [2022-10-20 06:10:38] [009-250][300/391] Time 0.09 (0.10) Data 0.00 (0.00) Base [Loss 2.331 (2.181)  Prec@1 4.69 (15.89) Prec@5 40.62 (65.99)]
*SEARCH* [2022-10-20 06:10:46] [009-250][390/391] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 2.312 (2.181)  Prec@1 7.50 (15.84) Prec@5 50.00 (65.69)]
[009-250] searching : loss=2.18, accuracy@1=15.84%, accuracy@5=65.69%, time-cost=415.8 s
[009-250] evaluate  : loss=2.19, accuracy@1=15.94%, accuracy@5=64.38%
[009-250] find-the-best : [Structure(4 nodes with |skip_connect~0|+|avg_pool_3x

*SEARCH* [2022-10-20 06:13:05] [012-250][000/391] Time 0.40 (0.40) Data 0.30 (0.30) Base [Loss 2.334 (2.334)  Prec@1 7.81 (7.81) Prec@5 46.88 (46.88)]
*SEARCH* [2022-10-20 06:13:18] [012-250][100/391] Time 0.14 (0.14) Data 0.00 (0.00) Base [Loss 2.332 (2.156)  Prec@1 7.81 (17.13) Prec@5 35.94 (66.58)]
*SEARCH* [2022-10-20 06:13:33] [012-250][200/391] Time 0.14 (0.14) Data 0.00 (0.00) Base [Loss 2.109 (2.150)  Prec@1 14.06 (17.48) Prec@5 71.88 (67.46)]
*SEARCH* [2022-10-20 06:13:47] [012-250][300/391] Time 0.12 (0.14) Data 0.00 (0.00) Base [Loss 2.293 (2.143)  Prec@1 12.50 (17.77) Prec@5 51.56 (68.16)]
*SEARCH* [2022-10-20 06:13:55] [012-250][390/391] Time 0.07 (0.13) Data 0.00 (0.00) Base [Loss 2.311 (2.141)  Prec@1 10.00 (17.88) Prec@5 47.50 (68.40)]
[012-250] searching : loss=2.14, accuracy@1=17.88%, accuracy@5=68.40%, time-cost=563.2 s
[012-250] evaluate  : loss=2.21, accuracy@1=14.95%, accuracy@5=61.42%
[012-250] find-the-best : [Structure(4 nodes with |none~0|+|skip_connect~0|none

*SEARCH* [2022-10-20 06:16:16] [015-250][000/391] Time 0.54 (0.54) Data 0.28 (0.28) Base [Loss 2.110 (2.110)  Prec@1 23.44 (23.44) Prec@5 71.88 (71.88)]
*SEARCH* [2022-10-20 06:16:31] [015-250][100/391] Time 0.14 (0.15) Data 0.00 (0.00) Base [Loss 2.319 (2.122)  Prec@1 9.38 (18.86) Prec@5 39.06 (68.97)]
*SEARCH* [2022-10-20 06:16:42] [015-250][200/391] Time 0.09 (0.13) Data 0.00 (0.00) Base [Loss 2.310 (2.137)  Prec@1 17.19 (18.07) Prec@5 48.44 (67.55)]
*SEARCH* [2022-10-20 06:16:52] [015-250][300/391] Time 0.14 (0.12) Data 0.00 (0.00) Base [Loss 2.070 (2.123)  Prec@1 21.88 (18.83) Prec@5 76.56 (68.89)]
*SEARCH* [2022-10-20 06:17:04] [015-250][390/391] Time 0.14 (0.12) Data 0.00 (0.00) Base [Loss 1.947 (2.116)  Prec@1 25.00 (18.90) Prec@5 75.00 (69.66)]
[015-250] searching : loss=2.12, accuracy@1=18.90%, accuracy@5=69.66%, time-cost=709.7 s
[015-250] evaluate  : loss=2.10, accuracy@1=20.13%, accuracy@5=70.62%
[015-250] find-the-best : [Structure(4 nodes with |avg_pool_3x3~0|+|avg_pool_

*SEARCH* [2022-10-20 06:19:06] [018-250][000/391] Time 0.40 (0.40) Data 0.30 (0.30) Base [Loss 2.001 (2.001)  Prec@1 23.44 (23.44) Prec@5 81.25 (81.25)]
*SEARCH* [2022-10-20 06:19:16] [018-250][100/391] Time 0.14 (0.10) Data 0.00 (0.00) Base [Loss 2.098 (2.107)  Prec@1 29.69 (19.51) Prec@5 75.00 (69.76)]
*SEARCH* [2022-10-20 06:19:25] [018-250][200/391] Time 0.09 (0.10) Data 0.00 (0.00) Base [Loss 2.039 (2.106)  Prec@1 25.00 (19.29) Prec@5 75.00 (69.65)]
*SEARCH* [2022-10-20 06:19:35] [018-250][300/391] Time 0.09 (0.10) Data 0.00 (0.00) Base [Loss 1.928 (2.110)  Prec@1 34.38 (19.08) Prec@5 82.81 (69.60)]
*SEARCH* [2022-10-20 06:19:43] [018-250][390/391] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 2.111 (2.111)  Prec@1 25.00 (18.99) Prec@5 72.50 (69.44)]
[018-250] searching : loss=2.11, accuracy@1=18.99%, accuracy@5=69.44%, time-cost=828.8 s
[018-250] evaluate  : loss=2.10, accuracy@1=19.80%, accuracy@5=69.72%
[018-250] find-the-best : [Structure(4 nodes with |avg_pool_3x3~0|+|none~0|a

*SEARCH* [2022-10-20 06:21:42] [021-250][000/391] Time 0.39 (0.39) Data 0.28 (0.28) Base [Loss 2.308 (2.308)  Prec@1 6.25 (6.25) Prec@5 48.44 (48.44)]
*SEARCH* [2022-10-20 06:21:55] [021-250][100/391] Time 0.14 (0.13) Data 0.00 (0.00) Base [Loss 1.929 (2.082)  Prec@1 25.00 (19.28) Prec@5 81.25 (70.70)]
*SEARCH* [2022-10-20 06:22:10] [021-250][200/391] Time 0.14 (0.14) Data 0.00 (0.00) Base [Loss 2.095 (2.082)  Prec@1 21.88 (19.39) Prec@5 81.25 (70.59)]
*SEARCH* [2022-10-20 06:22:24] [021-250][300/391] Time 0.14 (0.14) Data 0.00 (0.00) Base [Loss 2.305 (2.086)  Prec@1 6.25 (19.37) Prec@5 50.00 (70.40)]
*SEARCH* [2022-10-20 06:22:35] [021-250][390/391] Time 0.08 (0.14) Data 0.00 (0.00) Base [Loss 2.002 (2.086)  Prec@1 15.00 (19.48) Prec@5 77.50 (70.28)]
[021-250] searching : loss=2.09, accuracy@1=19.48%, accuracy@5=70.28%, time-cost=958.8 s
[021-250] evaluate  : loss=2.07, accuracy@1=20.10%, accuracy@5=71.72%
[021-250] find-the-best : [Structure(4 nodes with |none~0|+|none~0|skip_connect

*SEARCH* [2022-10-20 06:24:51] [024-250][000/391] Time 0.43 (0.43) Data 0.35 (0.35) Base [Loss 2.313 (2.313)  Prec@1 12.50 (12.50) Prec@5 48.44 (48.44)]
*SEARCH* [2022-10-20 06:24:59] [024-250][100/391] Time 0.07 (0.09) Data 0.00 (0.00) Base [Loss 1.910 (2.063)  Prec@1 23.44 (20.53) Prec@5 81.25 (71.70)]
*SEARCH* [2022-10-20 06:25:07] [024-250][200/391] Time 0.08 (0.09) Data 0.00 (0.00) Base [Loss 2.325 (2.070)  Prec@1 7.81 (20.72) Prec@5 46.88 (71.67)]
*SEARCH* [2022-10-20 06:25:16] [024-250][300/391] Time 0.13 (0.08) Data 0.00 (0.00) Base [Loss 2.045 (2.062)  Prec@1 20.31 (20.89) Prec@5 73.44 (72.13)]
*SEARCH* [2022-10-20 06:25:27] [024-250][390/391] Time 0.12 (0.09) Data 0.00 (0.00) Base [Loss 1.938 (2.060)  Prec@1 17.50 (20.74) Prec@5 82.50 (72.15)]
[024-250] searching : loss=2.06, accuracy@1=20.74%, accuracy@5=72.15%, time-cost=1090.0 s
[024-250] evaluate  : loss=2.06, accuracy@1=20.44%, accuracy@5=71.49%


In [7]:
torch.save(search_model.state_dict(), "./cell_wise_250ep_cosine.pth")

# Train a found model

In [14]:
print(best_arch)
import copy
best_arch_copy = copy.deepcopy(best_arch)

[Structure(4 nodes with |nor_conv_3x3~0|+|avg_pool_3x3~0|skip_connect~1|+|skip_connect~0|nor_conv_1x1~1|none~2|), Structure(4 nodes with |avg_pool_3x3~0|+|nor_conv_3x3~0|none~1|+|avg_pool_3x3~0|none~1|skip_connect~2|), Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|nor_conv_1x1~1|+|avg_pool_3x3~0|nor_conv_1x1~1|avg_pool_3x3~2|), Structure(4 nodes with |nor_conv_1x1~0|+|avg_pool_3x3~0|avg_pool_3x3~1|+|avg_pool_3x3~0|nor_conv_3x3~1|none~2|), Structure(4 nodes with |nor_conv_3x3~0|+|skip_connect~0|nor_conv_3x3~1|+|skip_connect~0|skip_connect~1|nor_conv_1x1~2|), Structure(4 nodes with |none~0|+|avg_pool_3x3~0|none~1|+|nor_conv_1x1~0|nor_conv_3x3~1|skip_connect~2|), Structure(4 nodes with |avg_pool_3x3~0|+|none~0|none~1|+|avg_pool_3x3~0|skip_connect~1|none~2|), Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_1x1~0|skip_connect~1|+|avg_pool_3x3~0|nor_conv_3x3~1|nor_conv_3x3~2|), Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|nor_conv_1x1~1|+|nor_conv_3x3~0|nor_conv_1x1~1|

In [15]:
print(args)
args.save_dir = '../results/NB201/test'

Namespace(arch_nas_dataset=None, channel=16, config_path='../configs/RANDOM.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=5, print_freq=100, rand_seed=16467, save_dir='../results/NB201/test', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [17]:
print(config)

Configure(scheduler='cos', eta_min=0.0, epochs=200, warmup=0, optim='SGD', LR=0.1, decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=256, class_num=10, xshape=(1, 3, 32, 32))


In [18]:
logger = prepare_logger(args)

cifar_train_config_path = "../configs/nas-benchmark/CIFAR.config"
###
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(cifar_train_config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": True, # true for eval
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))

network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)

start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Main Function with logger : Logger(dir=../results/NB201/test, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ../configs/RANDOM.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 100
rand_seed        : 16467
save_dir         : ../results/NB201/test
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already downloaded and verified
../configs/nas-benchmark/CIFAR.config
Configure(scheduler='cos', eta_min=0.0, epochs=200, warmup=0, optim='SGD', LR=0.1, decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', b

In [20]:
from custom_search_cells import NAS201SearchCell as SearchCell

i = 0
for cell in network.module.cells:
    if isinstance(cell, SearchCell):
        cell.arch_cache = best_arch[i]
        i = i+1
print(i)
print(len(best_arch))

15
15


In [21]:
def search_func_one_arch(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the weights
        ### sample a cell-wise randomized network
#         network.module.random_genotype_per_cell(True)
        ###
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg

def valid_func_one_arch(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

#             network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg

In [22]:
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(0, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func_one_arch(
        search_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func_one_arch(
        valid_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.close()


[Search the 000-200-th epoch] Time Left: [00:00:00], LR=0.1
*SEARCH* [2022-09-14 12:17:43] [000-200][000/098] Time 0.64 (0.64) Data 0.33 (0.33) Base [Loss 2.352 (2.352)  Prec@1 6.25 (6.25) Prec@5 50.78 (50.78)]
*SEARCH* [2022-09-14 12:17:53] [000-200][097/098] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 1.697 (1.801)  Prec@1 35.71 (31.54) Prec@5 89.88 (84.53)]
[000-200] searching : loss=1.80, accuracy@1=31.54%, accuracy@5=84.53%, time-cost=11.2 s
[000-200] evaluate  : loss=2.09, accuracy@1=30.11%, accuracy@5=79.44%
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 000-200-th epoch : find the highest validation accuracy : 30.11%.
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 001-200-th epoch] Time Left: [01:08:23], LR=0.09999383162408304
*SEARCH* [2022-09-14 12:18:03] [001-200][000/0

*SEARCH* [2022-09-14 12:20:20] [007-200][097/098] Time 0.22 (0.14) Data 0.00 (0.00) Base [Loss 0.900 (0.952)  Prec@1 69.64 (66.00) Prec@5 94.05 (97.09)]
[007-200] searching : loss=0.95, accuracy@1=66.00%, accuracy@5=97.09%, time-cost=100.6 s
[007-200] evaluate  : loss=1.26, accuracy@1=57.24%, accuracy@5=95.80%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 007-200-th epoch : find the highest validation accuracy : 57.24%.
Find ../results/NB201/test/checkpoint/seed-16467-best.pth exist, delete is at first before saving
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 008-200-th epoch] Time Left: [01

*SEARCH* [2022-09-14 12:23:05] [015-200][097/098] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 0.739 (0.723)  Prec@1 74.40 (74.56) Prec@5 98.21 (98.53)]
[015-200] searching : loss=0.72, accuracy@1=74.56%, accuracy@5=98.53%, time-cost=198.9 s
[015-200] evaluate  : loss=0.95, accuracy@1=68.21%, accuracy@5=97.28%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 015-200-th epoch : find the highest validation accuracy : 68.21%.
Find ../results/NB201/test/checkpoint/seed-16467-best.pth exist, delete is at first before saving
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 016-200-th epoch] Time Left: [00

*SEARCH* [2022-09-14 12:25:50] [023-200][097/098] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 0.575 (0.602)  Prec@1 78.57 (79.05) Prec@5 99.40 (98.95)]
[023-200] searching : loss=0.60, accuracy@1=79.05%, accuracy@5=98.95%, time-cost=298.1 s
[023-200] evaluate  : loss=0.84, accuracy@1=73.08%, accuracy@5=97.68%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 023-200-th epoch : find the highest validation accuracy : 73.08%.
Find ../results/NB201/test/checkpoint/seed-16467-best.pth exist, delete is at first before saving
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 024-200-th epoch] Time Left: [00

*SEARCH* [2022-09-14 12:28:46] [032-200][097/098] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 0.689 (0.524)  Prec@1 78.57 (81.56) Prec@5 98.21 (99.21)]
[032-200] searching : loss=0.52, accuracy@1=81.56%, accuracy@5=99.21%, time-cost=406.1 s
[032-200] evaluate  : loss=1.54, accuracy@1=59.82%, accuracy@5=96.43%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 033-200-th epoch] Time Left: [00:55:10], LR=0.09343157572190958
*SEARCH* [2022-09-14 12:28:56] [033-200][000/098] Time 0.54 (0.54) Data 0.43 (0.43) Base [Loss 0.486 (0.486)  Prec@1 83.59 (83.59) Prec@5 99.22 (99.22)]
*SEARCH* [2022-09-14 12:29:08] [033-200][097/098] Time 0.14 (0.13) Data 0.00 (0.00) Base [Loss 0.603 (0.515)  Prec@1 77.98 (

...

*SEARCH* [2022-09-14 13:11:36] [158-200][000/098] Time 0.47 (0.47) Data 0.31 (0.31) Base [Loss 0.056 (0.056)  Prec@1 97.66 (97.66) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:11:51] [158-200][097/098] Time 0.13 (0.15) Data 0.00 (0.00) Base [Loss 0.104 (0.058)  Prec@1 95.24 (98.16) Prec@5 100.00 (100.00)]
[158-200] searching : loss=0.06, accuracy@1=98.16%, accuracy@5=100.00%, time-cost=1989.2 s
[158-200] evaluate  : loss=0.57, accuracy@1=86.04%, accuracy@5=99.36%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 159-200-th epoch] Time Left: [00:16:59], LR=0.01001576707564547
*SEARCH* [2022-09-14 13:12:01] [159-200][000/098] Time 0.56 (0.56) Data 0.36 (0.36) Base [Loss 0.038 (0.038)  Prec@1 9

*SEARCH* [2022-09-14 13:14:46] [167-200][000/098] Time 0.46 (0.46) Data 0.32 (0.32) Base [Loss 0.033 (0.033)  Prec@1 99.61 (99.61) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:14:56] [167-200][097/098] Time 0.09 (0.11) Data 0.00 (0.00) Base [Loss 0.055 (0.034)  Prec@1 98.21 (99.01) Prec@5 100.00 (100.00)]
[167-200] searching : loss=0.03, accuracy@1=99.01%, accuracy@5=100.00%, time-cost=2103.0 s
[167-200] evaluate  : loss=0.55, accuracy@1=86.82%, accuracy@5=99.40%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 168-200-th epoch] Time Left: [00:09:10], LR=0.006184665997806821
*SEARCH* [2022-09-14 13:15:03] [168-200][000/098] Time 0.49 (0.49) Data 0.31 (0.31) Base [Loss 0.043 (0.043)  Prec@1 

[175-200] evaluate  : loss=0.53, accuracy@1=87.60%, accuracy@5=99.31%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth
<<<--->>> The 175-200-th epoch : find the highest validation accuracy : 87.60%.
Find ../results/NB201/test/checkpoint/seed-16467-best.pth exist, delete is at first before saving
copy the file from ../results/NB201/test/checkpoint/seed-16467-basic.pth into ../results/NB201/test/checkpoint/seed-16467-best.pth

[Search the 176-200-th epoch] Time Left: [00:07:55], LR=0.0035111757055874327
*SEARCH* [2022-09-14 13:17:52] [176-200][000/098] Time 0.43 (0.43) Data 0.31 (0.31) Base [Loss 0.017 (0.017)  Prec@1 99.61 (99.61) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:18:02] [176-200][097/098] Tim

*SEARCH* [2022-09-14 13:20:43] [184-200][097/098] Time 0.08 (0.10) Data 0.00 (0.00) Base [Loss 0.018 (0.015)  Prec@1 100.00 (99.69) Prec@5 100.00 (100.00)]
[184-200] searching : loss=0.02, accuracy@1=99.69%, accuracy@5=100.00%, time-cost=2314.9 s
[184-200] evaluate  : loss=0.54, accuracy@1=87.56%, accuracy@5=99.26%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 185-200-th epoch] Time Left: [00:04:35], LR=0.0013815039801161723
*SEARCH* [2022-09-14 13:20:52] [185-200][000/098] Time 0.58 (0.58) Data 0.47 (0.47) Base [Loss 0.012 (0.012)  Prec@1 100.00 (100.00) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:21:02] [185-200][097/098] Time 0.09 (0.11) Data 0.00 (0.01) Base [Loss 0.017 (0.015)  Pre

*SEARCH* [2022-09-14 13:23:21] [193-200][000/098] Time 0.48 (0.48) Data 0.32 (0.32) Base [Loss 0.010 (0.010)  Prec@1 100.00 (100.00) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-14 13:23:36] [193-200][097/098] Time 0.14 (0.16) Data 0.00 (0.00) Base [Loss 0.018 (0.013)  Prec@1 98.81 (99.79) Prec@5 100.00 (100.00)]
[193-200] searching : loss=0.01, accuracy@1=99.79%, accuracy@5=100.00%, time-cost=2420.9 s
[193-200] evaluate  : loss=0.54, accuracy@1=87.74%, accuracy@5=99.37%
Find ../results/NB201/test/checkpoint/seed-16467-basic.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/checkpoint/seed-16467-basic.pth
Find ../results/NB201/test/seed-16467-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/NB201/test/seed-16467-last-info.pth

[Search the 194-200-th epoch] Time Left: [00:02:12], LR=0.0002219017698460002
*SEARCH* [2022-09-14 13:23:43] [194-200][000/098] Time 0.42 (0.42) Data 0.31 (0.31) Base [Loss 0.009 (0.009)  Prec